In [1]:
# !pip install -q 'transformers[torch]' pyarrow pandas plotly datasets torch pillow
# !../data/prepare_data.sh


In [2]:
from typing import Callable, List

home_image_folders_str = """bathroom
bedroom
children_room
corridor
dining_room
garage
kitchen
laundry
livingroom
nursery
office
pantry
"""
home_room_folders = home_image_folders_str.split('\n')

from pathlib import Path
def enumerate_data_folder(folder: str) -> List[Path]:
    path = Path(f"../data/cvpr_raw/indoorCVPR_09/Images/{folder}")
    return list(path.glob("*"))

In [3]:
import torch
from PIL import Image
from transformers import pipeline

def get_fast_backend() -> str:
    if torch.backends.mps.is_available():
        return "mps"
    elif torch.backends.cuda.is_built():
        return "cuda"

# depth_estimator = pipeline("depth-estimation", model="vinvino02/glpn-nyu", device=get_fast_backend())
# depth_estimator.model.device

/Users/abenstirling/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# image = Image.open(enumerate_data_folder('kitchen')[0])
# predictions = depth_estimator(image)
# prediction = predictions['predicted_depth']
# depth = predictions['depth']
# depth

In [5]:
# depth: Image
# depth.save("sample_depth.png")

In [6]:
import arrow
import pandas
import plotly.express as px
from IPython.display import display

# Assuming 'output' is your depth data in native float format
# Make sure it's in CPU and numpy format for visualization
# If it's not already, you can do something like:
# output = prediction.squeeze().to('cpu').detach().numpy()

# Now, let's visualize with Plotly
# fig = px.imshow(prediction.squeeze().cpu().numpy(), color_continuous_scale='viridis')
# fig.update_layout(coloraxis_showscale=True)
# fig.update_traces(hoverinfo="x+y+z", hovertemplate="x: %{x}<br>y: %{y}<br>Depth: %{z}")

# display(fig.show())
# display(image)

In [7]:
from datasets import Dataset
from ipywidgets import IntProgress, Layout, HBox
from IPython.display import display
from transformers.pipelines.pt_utils import KeyDataset
import gc
import os


total_image_count = 0
for category in home_room_folders:
    files = [path for path in enumerate_data_folder(category) if os.path.isfile(path)]
    total_image_count += len(files)

if 'depth_estimator' in locals():
    del depth_estimator
!mkdir -p cvpr_depth/

f = IntProgress(min=0, max=total_image_count, description='Progress', layout=Layout(width='100%'))
display(f)

def make_process(folder):
    def process(chunk):
        estimator = pipeline("depth-estimation", model="vinvino02/glpn-nyu", device=get_fast_backend(), batch_size=15)
        print(f"Inferring for {chunk['name']}")
        res = estimator(chunk['image'])
        out = {'prediction': res}

        names = chunk['name']
        for output, name in zip(res, names):
            torch.save(output['predicted_depth'], f"cvpr_depth/{folder}/{name}.pt")
            f.value += 1
            f.description = f"{f.value}/{f.max}"
        

        torch.cuda.empty_cache()
        gc.collect()

        return out
    
    return process

for folder in home_room_folders:
    !mkdir -p ./cvpr_depth/{folder}

    # paths = [f for f in enumerate_data_folder(folder) if '8705' in f.name]
    paths = [path for path in enumerate_data_folder(folder) if os.path.isfile(path)]
    images = [Image.open(path) for path in paths]
    names = [path.name for path in paths]

    dataset = Dataset.from_dict({'image': images, 'name': names})
    print(f"Processing folder: {folder}")
    # result = dataset.map(make_processor(estimator))
    # result = estimator(KeyDataset(dataset, "image"))
    result_raw = dataset.map(make_process(folder), batched=True, batch_size=50)

    del paths
    del images
    del names
    del dataset
    del result_raw
    torch.cuda.empty_cache()
    gc.collect()


IntProgress(value=0, description='Progress', layout=Layout(width='100%'), max=3662)

Processing folder: bathroom


Parameter 'function'=<function make_process.<locals>.process at 0x175c2b820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Inferring for ['bath26.jpg', 'bathroom35.jpg', 'indoor_0333.jpg', 'img_2398.jpg', 'dscn2018.jpg', 'edificio_milano_ventas_alaria_034.jpg', 'indoor_0045.jpg', 'indoor_0078.jpg', 'IMG_3736.jpg', 'img_2399.jpg', 'indoor_0332.jpg', 'img_0029.jpg', 'IMG_2831.jpg', 'indoor_0124.jpg', 'bath31.jpg', 'indoor_0324.jpg', 'indoor_0278.jpg', 'indoor_0286.jpg', 'indoor_0279.jpg', 'IMG_3735.jpg', 'n190042.jpg', 'indoor_0137.jpg', 'bath303.jpg', 'd47.jpg', 'indoor_0309.jpg', 'dsc01582.jpg', 'IMG_1085.jpg', 'bath288.jpg', 'IMG_1084.jpg', 'roomscan34.jpg', 'indoor_0491.jpg', 'indoor_0452.jpg', 'b9.jpg', 'indoor_0493.jpg', 'IMG_2438.jpg', 'indoor_0242.jpg', 'IMG_5747.jpg', 'IMG_1253.jpg', 'IMG_1252.jpg', 'indoor_0082.jpg', 'IMG_5785.jpg', 'itoiletpaper.jpg', 'indoor_0531.jpg', 'indoor_0486.jpg', 'IMG_2439.jpg', 'indoor_0323.jpg', 'b8.jpg', 'bath.jpg', 'indoor_0135.jpg', 'IMG_1341.jpg']


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 256 but got size 512 for tensor number 2 in the list.

In [ ]:
test_depth = torch.load("cvpr_depth/bathroom/008.jpg.pt")